## Projet Matière en Traitement D'image

### Enhancement of handwritten text recognition using AI-based hybrid approach

**Objectif Principal :** Développer un système HTR qui s'adapte et apprend à partir de l'écriture manuscrite spécifique d'un utilisateur au fil du temps, améliorant ainsi la précision de la reconnaissance en utilisant la méthode Hybrid.

## Preprocessing Input Data and Labels

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv

2024-06-08 11:31:10.293433: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
import os
import cv2

import re

In [4]:
import string
import time

from keras.preprocessing.sequence import pad_sequences

from keras.layers import Dense, LSTM, Reshape, BatchNormalization, Input, Conv2D, MaxPool2D, Lambda, Bidirectional
from keras.models import Model
from keras.activations import relu, sigmoid, softmax
import keras.backend as K
from tensorflow.keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint

ImportError: cannot import name 'cast' from partially initialized module 'keras.src.backend' (most likely due to a circular import) (/opt/anaconda3/lib/python3.11/site-packages/keras/src/backend/__init__.py)

In [ ]:
char_encoding = [' ', '!', '"', '#', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
max_len_label = 0

def encode_to_labels(txt):
    # encoding each output word into digits
    dig_lst = []
    for index, char in enumerate(txt):
        try:
            dig_lst.append(char_encoding.index(char))
        except:
            print(char)
        
    return dig_lst

In [ ]:
valid_path = []
valid_label = []
valid_label_enc = []
valid_input_len = []
valid_label_len = []

with open("/content/RWTH.iam_word_gt_final.valid.thresh") as valid_list:
  for value in valid_list:
    valid_path.append(value.split(",")[0].strip()+".png")
    label = value.split(" ",1)[1].strip()
    valid_label.append(label)
    valid_label_enc.append(encode_to_labels(label))
    valid_label_len.append(len(label))
    valid_input_len.append(31)

print(valid_path[:5])

max_len_label = max(max_len_label,len(max(valid_label, key=len)))
# print(max_len_label)

In [ ]:
train_path = []
train_label = []
train_label_enc = []
train_input_len = []
train_label_len = []

with open("/content/RWTH.iam_word_gt_final.train.thresh") as train_list:
  for value in train_list:
    train_path.append(value.split(",")[0].strip()+".png")
    label = value.split(" ",1)[1].strip()
    train_label.append(label)
    train_label_enc.append(encode_to_labels(label))
    train_label_len.append(len(label))
    train_input_len.append(31)

print(train_label[:5])

max_len_label = max(max_len_label,len(max(train_label, key=len)))
# print(max_len_label)

In [ ]:
test_path = []
test_label = []
with open("/content/RWTH.iam_word_gt_final.test.thresh") as test_list:
  for value in test_list:
    test_path.append(value.split(",")[0].strip()+".png")
    test_label.append(value.split(" ",1)[1].strip())

# print(test_path)

max_len_label = max(max_len_label,len(max(test_label, key=len)))
# print(max_len_label)

In [ ]:
print(max_len_label)

In [ ]:
train_padded_txt = pad_sequences(train_label_enc, maxlen=max_len_label, padding='post', value = len(char_encoding))
valid_padded_txt = pad_sequences(valid_label_enc, maxlen=max_len_label, padding='post', value = len(char_encoding))

In [ ]:
train_img = []
valid_img = []
test_img = []
all_paths = {}

for root, dirs, files in os.walk("words/"):
  for file in files:
    all_paths[file]=os.path.join(root, file)


for file in train_path:
  if file in all_paths:
    input_img = cv2.cvtColor(cv2.imread(all_paths[file]),cv2.COLOR_BGR2GRAY)
    # input_img = cv2.imread(all_paths[file])
    # print(input_img.shape, type(input_img))
    # print(input_img)
    input_img = cv2.resize(input_img, (128,32))
    # print(input_img.shape)
    # print(input_img)
    input_img = np.expand_dims(input_img, axis=2)
    # input_img = cv2.resize(input_img, (128,32))
    # print(input_img.shape)
    input_img = input_img/255
    # print(input_img)
    train_img.append(input_img)

for file in valid_path:
  if file in all_paths:
    input_img = cv2.cvtColor(cv2.imread(all_paths[file]),cv2.COLOR_BGR2GRAY)
    input_img = cv2.resize(input_img, (128,32))
    input_img = np.expand_dims(input_img, axis=2)
    input_img = input_img/255
    valid_img.append(input_img)

for file in test_path:
  if file in all_paths:
    input_img = cv2.cvtColor(cv2.imread(all_paths[file]),cv2.COLOR_BGR2GRAY)
    input_img = cv2.resize(input_img, (128,32))
    input_img = np.expand_dims(input_img, axis=2)
    input_img = input_img/255
    test_img.append(input_img)

  # elif file in valid_path:
  #   valid_img.append(cv2.imread(os.path.join(root, file)))

  # elif file in test_path:
  #   test_img.append(cv2.imread(os.path.join(root, file)))

print(test_img[0].shape)
print(len(train_img), len(test_img), len(valid_img))

In [ ]:
from google.colab.patches import cv2_imshow
cv2_imshow(train_img[0]*255)
print(train_label[0])

cv2_imshow(valid_img[0]*255)
print(valid_label[0])

cv2_imshow(test_img[0]*255)
print(test_label[0])